# Module 5 Section 1 Lab 1 - Ensemble Learning
Semua metode memiliki kekurangan, seperti kurang stabil dan mudah terjebak pada kondisi *overfit* ketika digunakan pada suatu kasus tertentu yang kurang sesuai dengan karakter metode tersebut. Oleh karena itu, sejumlah peneliti mengusulkan teknik pembelajaran gabungan atau *Ensemble Learning*. 

## Ide dan Motivasi
Misalkan terdapat tiga nilai prediksi harga emas untuk hari besok (naik, naik dan turun) yang dikeluarkan oleh tiga Lembaga berbeda (A, B, dan C). Ketiga lembaga tersebut membangun model prediksi menggunakan data dan teknik independen (tidak ada keterkaitan antar lembaga), di mana akurasi prediksi selama setahun terkahir adalah 80%, 82%, dan 85% secara berturut-turut untuk Lembaga A, B, dan C.

Prediksi harga emas mana yang Anda bisa percaya?<br>
Apakah Anda lebih percaya pada Lembaga C karena selama setahun ini memberikan akurasi paling tinggi sebesar 85%? Jika Ya, berarti kita yakin bahwa besok hari harga emas akan turun.

Jika saya akan lebih percaya pada suara terbayak (*majority vote*) dari ketiga prediksi tersebut. Artinya, saya lebih percaya bahwa besok hari harga emas akan naik karena dua dari tiga Lembaga memprediksi naik. Jika Lembaga A, B, dan C saling independen, maka secara teori probabilitas penggabungkan ketiga model akan memberikan akurasi yang lebih besar dibandingkan dengan akurasi masing-masing model jika berdiri sendiri. Perhitungan akurasi prediksi gabungan didapatkan dari tiga kejadian, di mana dua suara 'naik' dan satu suara 'turun'. ditambah dengan satu kejadian di mana ketiga suara sama semua (mufakat).

Dengan demikian, masuk akal jika lebih percaya pada suara terbayak dari ketiga Lembaga tersebut karena akurasi (peluang kebenarannya) jauh lebih besar jika dibanding akurasi masing-masing Lembaga yang berdiri sendiri.

Bagaimanapun, konsep *majority voting* inihanya berlaku jika setiap model saling independen, yang dilatih dengan himpunan data yang saling terpisah. Jika setiap model tidak saling independen, makan konsep *majority voting* tidak akan bekerja.

Baca selanjutnya: https://link.springer.com/chapter/10.1007/3-540-45014-9_1

## Cara Kerja Ensemble Learning?
Setiap algoritma *Ensemble Learning* terdiri dari dua langkah utama:
1. Menghasilkan kelompok prediksi menggunakan algoritma pembelajaran sederhana.
2. Menggabungkan prediksi menjadi satu model 'agregat'.

Ensemble dapat dicapai melalui beberapa teknik.




# Voting Ensembles
*Voting ensemble* atau dikenal dengan *majority voting ensemble* adalah model pembelajaran dengan menggabungkan prediksi dari beberapa model. Kita dapat menggunakannya untuk tugas regresi maupun klasifikasi jika memiliki dua atau lebih model yang memiliki performa baik. Dalam tugas regresi, hasil akhir didapatkan dengan menghitung rata-rata setiap prediksi model. Sedangkan dalam tugas klasifikasi, prediksi untuk setiap label dijumlahkan dan label dengan suara terbanyak diprediksi. Terdapat dua pendekatan yang dapat digunakan pada tugas klasifikasi:
- *Hard voting* memprediksi kelas dengan jumlah suara terbanyak dari model.
- *Soft voting* memprediksi kelas dengan probabilitas penjumlahan terbesar dari model.

Baca selanjutnya: http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier


In [24]:
import numpy as np
import pandas as pd

df = pd.read_csv('dataset/loan_prediction.csv')
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status
0,5849,0.0,0.0,360.0,1.0,1
1,4583,1508.0,128.0,360.0,1.0,0
2,3000,0.0,66.0,360.0,1.0,1
3,2583,2358.0,120.0,360.0,1.0,1
4,6000,0.0,141.0,360.0,1.0,1


In [25]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [26]:
'''
Soft Voting/Majority Rule classifier for unfitted estimators
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html
'''

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Buat model indiviual
logreg_clf = LogisticRegression()
decision_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier()


In [30]:
# Implementasi Hard Voting
voting_clf_hard = VotingClassifier(estimators=[('Logistic Regression', logreg_clf), ('Decision Tree', decision_clf), ('k-NN', knn_clf)], voting='hard')
voting_clf_hard.fit(X_train, y_train)

# Performa model
y_pred_hard = voting_clf_hard.predict(X_test)
accuracy_hard = accuracy_score(y_test, y_pred_hard)

In [31]:
# Implementasi Soft Voting
voting_clf_soft = VotingClassifier(estimators=[('Logistic Regression', logreg_clf), ('Decision Tree', decision_clf), ('k-NN', knn_clf)], voting='soft')
voting_clf_soft.fit(X_train, y_train)

# Performa model
y_pred_soft = voting_clf_soft.predict(X_test)
accuracy_soft = accuracy_score(y_test, y_pred_soft)

In [33]:
print('Hard voting accuracy:', accuracy_hard)
print('Soft voting accuracy:', accuracy_soft)

Hard voting accuracy: 0.6918918918918919
Soft voting accuracy: 0.6864864864864865


# Bagging (Bootstrap Aggregating) Ensemble
Merupakan metode *Ensemble Learning* efektif yang pertama kali diusulkan. *Bagging* dikenal sebagai salah satu metode yang paling simple dalam *Arching (Adaptive Reweighting and Combining)*, sebuah terminologi umum yang mengacu pada penggunaan kembali atau pemilihan data untuk meningkatkan akurasi klasifikasi.

*Bagging* membentuk *instance* yang membangun beberapa estimator pada himpunan data latih secara acak dan kemudian menggabungkan masing-masing prediksi model untuk membentuk prediksi akhir. *Bagging* menggunakan satu tipe algoritma pembelajaran dengan melibatkan manipulasi data latih dengan *resampling*. *Bagging* mempelajari $k$ algoritma pembelajaran pada $k$ sampel data latih yang berbeda.

Sampel dibuat secara independen dengan melakukan sampel ulang pada data latih menggunakan bobot yang seragam. Ini berarti bahwa pengambilan sampel titik data terjadi dengan penggantian. Proses pengambilan sampel dengan penggantian disebut *Bootstrapping*.

Semua *instance* dapat dilatih secara parallel melalui CPU core yang berbeda atau bahkan server yang berbeda. Demikian pula dengan prediksi dapat dibuat secara parallel. Inilah salah satu alasan mengapa metode *Bagging* sangat populer.

Baca selanjutnya: https://link.springer.com/article/10.1023/A:1018054314350

<center><img src='https://machinelearningmastery.com/wp-content/uploads/2020/11/Bagging-Ensemble.png' width=400></img><br><a href='https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/'>Sumber Gambar</a></center>


In [42]:
from sklearn.tree import DecisionTreeClassifier

decision_clf = DecisionTreeClassifier()

# Fitting single Decision Tree
decision_clf.fit(X_train, y_train)

y_pred_decision = decision_clf.predict(X_test)
score_dt = accuracy_score(y_test, y_pred_decision)


In [49]:
'''
A Bagging classifier is an ensemble meta-estimator that fits base classifiers each on random subsets of the original dataset and then aggregate their individual predictions.
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
'''

from sklearn.ensemble import BaggingClassifier

# Fitting bagging classifier with Decision Tree
bagging_clf = BaggingClassifier(base_estimator=decision_clf)
bagging_clf.fit(X_train, y_train)

y_pred_bagging = bagging_clf.predict(X_test)
score_bc_lr = accuracy_score(y_test, y_pred_bagging)

In [50]:
print('Single Decision Tree:', score_dt)
print('Bagging with Decision Tree accuracy:', score_bc_lr)

Single Decision Tree: 0.6702702702702703
Bagging with Decision Tree accuracy: 0.745945945945946


## Random Forest
Random Forest adalah algoritma bagging dengan pohon keputusan sebagai classifier/regressor dasar. Alih-alih mencari fitur terbaik saat memisahkan sebuah node, *random forest* mencari fitur terbaik di antara subset fitur yang acak. Ini menghasilkan keragaman pohon yang lebih besar.

Baca selanjutnya: https://journals.sagepub.com/doi/full/10.1177/1536867X20909688

<center><img src='https://miro.medium.com/max/700/1*5dq_1hnqkboZTcKFfwbO9A.png' width=800></img><br><a href='https://medium.com/analytics-vidhya/machine-learning-decision-trees-and-random-forest-classifiers-81422887a544'>Sumber Gambar</a></center>

In [73]:
'''
A random forest is a meta estimator that fits a number of decision tree classifiers on 
various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
'''

from sklearn.ensemble import RandomForestClassifier

ranfor_clf_1 = RandomForestClassifier(min_samples_leaf=2)
ranfor_clf_1.fit(X_train,y_train)

ranfor_clf_2 = RandomForestClassifier(max_leaf_nodes=5)
ranfor_clf_2.fit(X_train,y_train)

# Performa
y_pred_ranfor_1 = ranfor_clf_1.predict(X_test)
score_ranfor_1 = accuracy_score(y_test, y_pred_ranfor_1)

y_pred_ranfor_2 = ranfor_clf_2.predict(X_test)
score_ranfor_2 = accuracy_score(y_test, y_pred_ranfor_2)


In [75]:
print('Random forest accuracy with min_sample_leaf 2:', score_ranfor_1)
print('Random forest accuracy with max_leaf_nodes 5:', score_ranfor_2)

Random forest accuracy with min_sample_leaf 2: 0.7405405405405405
Random forest accuracy with max_leaf_nodes 5: 0.7567567567567568


# Stacking Ensemble
Pertama kali diusulkan untuk meminimasilir tingkat kesalahan generalisas dari satu atau lebih model. *Stacking* menggabungkan beberapa model klasifikasi melalui *meta-classifier*. Ini didasarkan pada ide sederhana: alih-alih menggunakan fungsi untuk menggabungkan prediksi semua algoritma pembelajaran dalam ansambel, kita melatih model untuk melakukan agregasi ini. Singkatnya, *Stacking* memvariasikan tipe model yang sesuai dengan data latih dan menggunakan model tersebut untuk menggabungkan prediksi.

*meta-classifier*: https://www.igi-global.com/dictionary/meta-classifier/45744<br>
Baca selanjutnya: https://www.sciencedirect.com/science/article/abs/pii/S0893608005800231

<center><img src='https://machinelearningmastery.com/wp-content/uploads/2020/11/Stacking-Ensemble.png' width=400></img><br><a href='https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/'>Sumber Gambar</a></center>

In [53]:
'''
Stacked generalization consists in stacking the output of individual estimator and use a classifier to compute the final prediction. 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html
'''

from sklearn.ensemble import StackingClassifier

estimators = [
    ('log_reg', LogisticRegression()),
    ('dt', DecisionTreeClassifier())
]

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train, y_train)

y_pred_bagging = stacking_clf.predict(X_test)
score_sc_lr = accuracy_score(y_test, y_pred_bagging)

In [54]:
print('Stacking accuracy:', score_sc_lr)

Stacking accuracy: 0.7297297297297297


# Boosting Ensemble
Sesuai dengan namanya, metode *Boosting* bekerja dengan cara memperkuat (*boost*) sebuah model pengklasifikasi. *Boosting* muncul dari ide apakah algoritma pembelajaran yang lemah dapat dimodifikasi secara berurutan menjadi lebih baik. *Boosting* mengacu pada metode Ensemble apa pun yang dapat menggabungkan algoritma pembelajaran yang lemah menjadi model yang kuat. Ide umum dari sebagian besar metode ini adalah melatih prediktor secara berurutan, masing-masing mencoba memperbaiki pendahulunya. Proses penguatan model secara sekuensial dilakukan sebanyak $T$ kali sampai dihasilkan model yang cukup kuat. Sejumlah $T$ model yang dihasilkan selanjutnya digabungkan menggunakan *majority vote* dengan pembobotan sesuai performanya.

Baca selanjutnya: https://www-ai.cs.tu-dortmund.de/LEHRE/PG/PG445/literatur/schapire_99a.pdf

<center><img src='https://machinelearningmastery.com/wp-content/uploads/2020/11/Boosting-Ensemble.png' width=400></img><br><a href='https://machinelearningmastery.com/tour-of-ensemble-learning-algorithms/'>Sumber Gambar</a></center>

Ada banyak jenis *Boosting*, namun yang paling populer adalah:
* AdaBoost (Adaptive Boosting)
* Gradient Boosting

## AdaBoost

Baca selanjutnya: https://cseweb.ucsd.edu/~yfreund/papers/IntroToBoosting.pdf

In [61]:
'''
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

'''

from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1))
ada_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)
score_ada = accuracy_score(y_test, y_pred_ada)

In [62]:
print('AdaBoost accuracy:', score_ada)

AdaBoost accuracy: 0.745945945945946


## Gradient Boosting

Baca selanjutnya: https://statweb.stanford.edu/~jhf/ftp/trebst.pdf

In [63]:
'''
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
'''

from sklearn.ensemble import GradientBoostingClassifier


gb_clf = GradientBoostingClassifier(n_estimators=200, min_samples_leaf=10, max_depth=1)
gb_clf.fit(X_train, y_train)

y_pred_gb = gb_clf.predict(X_test)
score_gb = accuracy_score(y_test, y_pred_gb)

In [64]:
print('Gradient Boosting accuracy:', score_gb)

Gradient Boosting accuracy: 0.7567567567567568
